# Autonomous Perception Robustness Testing Framework (APRTF)
### Development Journal

We show that our general framework can be used on the [NuScenes](https://www.nuscenes.org/) dataset using a multi-stage analysis proposed in ["Perception robustness testing at different levels of generality"](https://www.journalfieldrobotics.org/FR/Papers_files/10_Pezzementi.pdf).

In [ ]:
import os
import numpy as np
import torch

# augmentation
import torchvision.transforms as T
import aprtf.augmentations as A

# dataset
from nuscenes.utils.geometry_utils import view_points
from nuscenes import NuScenes
data_dir = './data/sets/nuScenes'
nusc = NuScenes(version='v1.0-mini', dataroot=data_dir, verbose=True)

# torchvision reference code
import aprtf.dataset as D
from aprtf.torchvision_detection.coco_utils import get_coco_api_from_dataset
from aprtf.pycocotools_robustness.cocoeval import COCOeval
from aprtf.torchvision_detection.coco_eval import CocoEvaluator
from aprtf.config import cfg
from aprtf.models import ModelBuilder
from aprtf.torchvision_detection import utils
#from aprtf.analysis import Analyzer

# logging
from tqdm import tqdm

print("All packages imported!")


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print("Random seed set")

## I. Pedestrian Detection

### Data and Labels

Time-ordered iterator of images and bounding boxes.

In [ ]:
# nuScenes
def box2bb(box, cam_intrinsic):
    corners = torch.tensor(view_points(box.corners(), view=cam_intrinsic, normalize=True)[:2, :])
    bb = torch.cat([torch.min(corners, dim=1).values, torch.max(corners, dim=1).values]).tolist()
    return bb
    
category = 'pedestrian'
sensor = 'CAM_FRONT'
visibility_threshold = 2

odgt = []

for scene in nusc.scene:
    next_sample_token = scene['first_sample_token']
    while next_sample_token:
        sample = nusc.get('sample', next_sample_token)
        sample_data = nusc.get('sample_data', sample['data'][sensor])

        # image filepaths
        sample_data_fp = os.path.join(data_dir,sample_data['filename'])

        # bounding boxes
        sample_data_bbs = []
        for ann in sample['anns']:
            _, box, cam_intrinsic = nusc.get_sample_data(sample['data'][sensor], selected_anntokens=[ann])
            if len(box) > 1:
                raise ValueError('more than one annotation')

            visibility_token = nusc.get('sample_annotation', ann)['visibility_token']
            visibility = int(visibility_token)
            if (len(box) == 1) and (category in box[0].name) and (visibility >= visibility_threshold):
                bb = box2bb(box[0], cam_intrinsic)
                sample_data_bbs.append(bb)

        # odgt
        odgt.append(
            {
                'image': sample_data_fp,
                'annotations': sample_data_bbs
            }
        )

        # next sample
        next_sample_token = sample['next']

normal_transform = D.get_transform(train=False) 
aug_transform = T.GaussianBlur(5,3)
all_transform = A.TransformAugmentationCompose(normal_transform, aug_transform)

dataset = D.PedestrianDetectionDataset(odgt, transforms=all_transform)

config = 'retinanet_resnet50_fpn-pennfudanped'
cfg_path = os.path.join('ckpt', config, 'config.yaml')
cfg.merge_from_file(cfg_path)


In [ ]:
config = 'retinanet_resnet50_fpn-pennfudanped'
cfg_path = os.path.join('ckpt', config, 'config.yaml')
cfg.merge_from_file(cfg_path)

#dataset_path = os.path.join('data','sets', 'PennFudanPed', cfg.DATASET.LIST.val)
#dataset = D.PedestrianDetectionDataset(dataset_path, transforms=all_transform)

In [ ]:
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=1, shuffle=False, num_workers=1,
 collate_fn=utils.collate_fn)

### Model

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    logging.info('No GPU found! Training on CPU')
    device = torch.device('cpu')

weights_path = os.path.join('ckpt', config, 'weights_best.pth')
model = ModelBuilder.build_detector(args=cfg.MODEL, weights=weights_path)
model.to(device)
model.eval()

### Metric

Recall and $FPR_A$.

In [ ]:
def evaluate(model, data_loader, device):
    cpu_device = torch.device("cpu")
    model.eval()
    coco = get_coco_api_from_dataset(data_loader.dataset)
    coco_evaluator = CocoEvaluator(COCOeval, coco, ['bbox'], score_min=0.9)

    for images, targets in tqdm(data_loader):
        images = list(img.to(device) for img in images)
        with torch.no_grad():
            outputs = model(images)
            outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        del images
        # you need to do .item() because an int is not treated the same as a tensor int
        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        coco_evaluator.update(res)

    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    return coco_evaluator

In [ ]:
evaluate_log = evaluate(model, data_loader, device)